 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500 Index.

In [14]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp500_history.csv`: Contains historical closing prices of the S&P 500 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [ ]:
# Reading whale returns
whale_returns = pd.read_csv("whale_returns.csv", index_col="Date", parse_dates = True)
whale_returns.head()

In [ ]:
# Count nulls
whale_returns.isnull().sum()

In [ ]:
# Drop nulls
whale_returns.dropna(inplace = True)
whale_returns.isnull().sum()

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns
algo_returns = pd.read_csv("algo_returns.csv", index_col="Date", parse_dates = True)
algo_returns.head(

In [ ]:
# Count nulls
algo_returns.isnull().sum()

In [ ]:
# Drop nulls
algo_returns = algo_returns.dropna()
algo_returns.head()

## S&P 500 Returns

Read the S&P 500 historic closing prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P 500 Closing Prices
sp500_history = pd.read_csv("sp500_history.csv", index_col="Date", parse_dates = True)
sp500_history.head()

In [ ]:
# Check Data Types
sp500_history.info()

In [ ]:
# Fix Data Types
sp500_history.head()
sp500_history["Close"]= sp500_history["Close"].str.replace("$"," ")
sp500_history.head()
sp500_history["Close"] = sp500_history["Close"].astype("float")

In [ ]:
# Calculate Daily Returns
sp500_history["Close"].pct_change()
sp500_history["Close"]= sp500_history["Close"].pct_change()
sp500_history.head()

In [ ]:
# Drop nulls
sp500_history = sp500_history.dropna()
sp500_history.head()

In [ ]:
# Rename `Close` Column to be specific to this portfolio.
sp500_history.columns=["S&P Returns"]
sp500_history.head()

## Combine Whale, Algorithmic, and S&P 500 Returns

In [ ]:
# Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single DataFrame with columns for each portfolio's returns.
combined_return = pd.concat([whale_returns, algo_returns, sp500_history],axis="columns", join="inner")
combined_return.head()

---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [ ]:
# Plot daily returns of all portfolios
combined_return.plot(figsize=(8,6))

#### Calculate and Plot cumulative returns.

In [ ]:
# Calculate cumulative returns of all portfolios
cumulative_return = (1+combined_return).cumprod()
# Plot cumulative returns
cumulative_return.plot(figsize=(8,6))

---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

### Create a box plot for each portfolio


In [ ]:
# Box plot to visually show risk
whale_returns.boxplot(figsize=(8,10))
algo_returns.boxplot(figsize=(8,10))
sp500_history.boxplot(figsize=(8,10))

### Calculate Standard Deviations

In [ ]:
# Calculate the daily standard deviations of all portfolios
stand_dev_calcs = combined_return.std()
stand_dev_calcs.head(6)

### Determine which portfolios are riskier than the S&P 500

In [ ]:
# Calculate  the daily standard deviation of S&P 500
sp500_history.std()
# Determine which portfolios are riskier than the S&P 500
print("The riskier portfolios are Tiger Global Management and Berkshire Hathaway, Inc.")

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
annualized_deviation = stand_dev_calcs.var() *252
np.sqrt(annualized_deviation)
print("The annualized standard deviation is" + str(annualized_deviation))

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for the S&P 500 using a 21-day window
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P 500
3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [ ]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window
rolling_deviation = combined_return.rolling(window=21).std()
# Plot the rolling standard deviation
rolling_deviation.plot(figsize=(10,10))

### Calculate and plot the correlation

In [ ]:
# Calculate the correlation
correlation = combined_return.corr()
# Display de correlation matrix
correlation

### Calculate and Plot Beta for a chosen portfolio and the S&P 500

In [ ]:
# Calculate covariance of a single portfolio
whale_returns_covariance = combined_return["Algo 2"].rolling(window=21).cov(combined_return["Whale Returns"])
whale_returns_variance = combined_return["Algo 2"].rolling(window=21).var()
# Calculate variance of S&P 500
sp500_covariance = combined_return["Algo 2"].rolling(window=21).cov(combined_return["S&P Returns"])
sp500_variance = combined_return["Algo 2"].rolling(window=21).var()
# Computing beta
algo2_beta = sp500_covariance / sp500_variance
# Plot beta trend
algo2_beta.plot(figsize = (10,12))

## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half-life.

In [ ]:
# Use `ewm` to calculate the rolling window



---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Annualized Sharpe Ratios
sharpe_ratios = combined_return - combined_return / stand_dev_calcs
sharpe_ratios

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios.plot(figsize= (8,10))

### Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

Write your answer here! = Tiger Global outperforms both

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

For this demo solution, we fetch data from three companies listes in the S&P 500 index.

* `GOOG` - [Google, LLC](https://en.wikipedia.org/wiki/Google)

* `AAPL` - [Apple Inc.](https://en.wikipedia.org/wiki/Apple_Inc.)

* `COST` - [Costco Wholesale Corporation](https://en.wikipedia.org/wiki/Costco)

In [5]:
!pip install yfinance
import yfinance as yf

  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8500 sha256=b5b57c1199313937c40ed25c54978e578008b81c6fbf0978be2f88c9d72f892a
  Stored in directory: c:\users\laree\appdata\local\pip\cache\wheels\f2\b5\2c\59ba95dcf854e542944c75fe3da584e4e3833b319735a0546c
Successfully built multitasking


In [6]:
# Reading data from 1st stock
goog_df = yf.download('GOOG',start='2020-12-31',end='2021-12-31',progress=False,)
goog_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-31,1735.420044,1758.930054,1735.420044,1751.880005,1751.880005,1011900
2021-01-04,1757.540039,1760.650024,1707.849976,1728.239990,1728.239990,1901900
2021-01-05,1725.000000,1747.670044,1718.015015,1740.920044,1740.920044,1145300
2021-01-06,1702.630005,1748.000000,1699.000000,1735.290039,1735.290039,2602100
2021-01-07,1740.060059,1788.400024,1737.050049,1787.250000,1787.250000,2265000


In [7]:
# Reading data from 2nd stock
aapl_df = yf.download('AAPL',start='2020-12-31',end='2021-12-31',progress=False,)
aapl_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-31,134.080002,134.740005,131.720001,132.690002,131.709198,99116600
2021-01-04,133.520004,133.610001,126.760002,129.410004,128.453461,143301900
2021-01-05,128.889999,131.740005,128.429993,131.009995,130.041611,97664900
2021-01-06,127.720001,131.050003,126.379997,126.599998,125.664215,155088000
2021-01-07,128.360001,131.630005,127.860001,130.919998,129.952286,109578200


In [8]:
# Reading data from 3rd stock
cost_df = yf.download('COST',start='2020-12-31',end='2021-12-31',progress=False,)
cost_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-31,374.989990,377.040009,372.459991,376.779999,373.360840,1628900
2021-01-04,377.429993,381.549988,374.809998,380.149994,376.700226,3322200
2021-01-05,378.859985,380.209991,375.250000,375.739990,372.330292,2748400
2021-01-06,373.820007,375.799988,369.649994,370.019989,366.662140,2661400
2021-01-07,371.010010,371.649994,366.790009,367.920013,364.581238,2849500


In [11]:
# Combine all stocks in a single DataFrame
stocks_return = pd.concat([goog_df, aapl_df, cost_df],axis="columns", join="inner")
stocks_return.head()

,Open,High,Low,Close,Adj Close,Volume,Open,High,Low,Close,Adj Close,Volume,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,,,,,,,,,,,,
2020-12-31,1735.420044,1758.930054,1735.420044,1751.880005,1751.880005,1011900,134.080002,134.740005,131.720001,132.690002,131.709198,99116600,374.989990,377.040009,372.459991,376.779999,373.360840,1628900
2021-01-04,1757.540039,1760.650024,1707.849976,1728.239990,1728.239990,1901900,133.520004,133.610001,126.760002,129.410004,128.453461,143301900,377.429993,381.549988,374.809998,380.149994,376.700226,3322200
2021-01-05,1725.000000,1747.670044,1718.015015,1740.920044,1740.920044,1145300,128.889999,131.740005,128.429993,131.009995,130.041611,97664900,378.859985,380.209991,375.250000,375.739990,372.330292,2748400
2021-01-06,1702.630005,1748.000000,1699.000000,1735.290039,1735.290039,2602100,127.720001,131.050003,126.379997,126.599998,125.664215,155088000,373.820007,375.799988,369.649994,370.019989,366.662140,2661400
2021-01-07,1740.060059,1788.400024,1737.050049,1787.250000,1787.250000,2265000,128.360001,131.630005,127.860001,130.919998,129.952286,109578200,371.010010,371.649994,366.790009,367.920013,364.581238,2849500


In [12]:
# Reset Date index
stocks_return.reset_index("Date") 
stocks_return.head()

,Open,High,Low,Close,Adj Close,Volume,Open,High,Low,Close,Adj Close,Volume,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,,,,,,,,,,,,
2020-12-31,1735.420044,1758.930054,1735.420044,1751.880005,1751.880005,1011900,134.080002,134.740005,131.720001,132.690002,131.709198,99116600,374.989990,377.040009,372.459991,376.779999,373.360840,1628900
2021-01-04,1757.540039,1760.650024,1707.849976,1728.239990,1728.239990,1901900,133.520004,133.610001,126.760002,129.410004,128.453461,143301900,377.429993,381.549988,374.809998,380.149994,376.700226,3322200
2021-01-05,1725.000000,1747.670044,1718.015015,1740.920044,1740.920044,1145300,128.889999,131.740005,128.429993,131.009995,130.041611,97664900,378.859985,380.209991,375.250000,375.739990,372.330292,2748400
2021-01-06,1702.630005,1748.000000,1699.000000,1735.290039,1735.290039,2602100,127.720001,131.050003,126.379997,126.599998,125.664215,155088000,373.820007,375.799988,369.649994,370.019989,366.662140,2661400
2021-01-07,1740.060059,1788.400024,1737.050049,1787.250000,1787.250000,2265000,128.360001,131.630005,127.860001,130.919998,129.952286,109578200,371.010010,371.649994,366.790009,367.920013,364.581238,2849500


In [13]:
# Reorganize portfolio data by having a column per symbol
sorted(stocks_return)

['Adj Close',
 'Adj Close',
 'Adj Close',
 'Close',
 'Close',
 'Close',
 'High',
 'High',
 'High',
 'Low',
 'Low',
 'Low',
 'Open',
 'Open',
 'Open',
 'Volume',
 'Volume',
 'Volume']

In [ ]:
# Calculate daily returns
daily_returns = portfolio.pct_change()

# Drop NAs
daily_returns = daily_returns.dropna().copy()

# Display sample data
daily_returns.head()

## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return
portfolio_returns = daily_returns.dot(weights)

# Display sample data
portfolio_returns.head()

## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Join your returns DataFrame to the original returns DataFrame
all_portfolios["Custom"] = portfolio_returns

In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
all_portfolios = all_portfolios.dropna().copy()

## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [ ]:
# Calculate the annualized `std`
astd = all_portfolios.astd() * np.sqrt(252)

### Calculate and plot rolling `std` with 21-day window

In [ ]:
# Calculate rolling standard deviation
rolling_std_dev = all_portfolios.rolling(window=21).std()

# Plot rolling standard deviation
rolling_std_dev.plot(figsize=(20, 10), title="21 Day Rolling STD")

### Calculate and plot the correlation

In [ ]:
# Calculate and plot the correlation
df = all_portfolios.corr()
df.style.background_gradient(cmap="bwr")

### Calculate and Plot Rolling 60-day Beta for Your Portfolio compared to the S&P 500

In [ ]:
# Calculate and plot Beta
covariance = all_portfolios["Custom"].rolling(window=60).cov(all_portfolios["S&P 500"])
variance = all_portfolios["S&P 500"].rolling(60).var()
beta = (covariance / variance)
beta.plot(figsize=(20, 10), title="Cstm Beta")

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [ ]:
# Calculate Annualzied Sharpe Ratios
sharpe_ratios = (all_portfolios.mean() * 252) / (all_portfolios.std() * np.sqrt(252))

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios.plot(kind="bar", title="Sharpe Ratios")

### How does your portfolio do?

Write your answer here!